In [ ]:
import tkinter as tk
from tkinter import messagebox, filedialog
import sqlite3
from PIL import Image
import os
import speech_recognition as sr

# Database setup
conn = sqlite3.connect("tasks.db")
cursor = conn.cursor()
cursor.execute("""
    CREATE TABLE IF NOT EXISTS tasks (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        task TEXT NOT NULL,
        completed BOOLEAN NOT NULL DEFAULT 0,
        image_path TEXT
    )
""")
conn.commit()

def add_task(task_text=None):
    if task_text is None:  # If called without speech input
        task_text = task_entry.get().strip()

    if task_text:
        cursor.execute("INSERT INTO tasks (task) VALUES (?)", (task_text,))
        conn.commit()
        task_entry.delete(0, tk.END)
        refresh_tasks()
    else:
        messagebox.showwarning("Warning", "Task cannot be empty")

def delete_task():
    selected_item = task_listbox.curselection()
    if selected_item:
        task_id = int(task_listbox.get(selected_item).split(" - ")[0])
        cursor.execute("DELETE FROM tasks WHERE id = ?", (task_id,))
        conn.commit()
        refresh_tasks()
    else:
        messagebox.showwarning("Warning", "Select a task to delete")

def mark_completed():
    selected_item = task_listbox.curselection()
    if selected_item:
        task_id = int(task_listbox.get(selected_item).split(" - ")[0])
        cursor.execute("UPDATE tasks SET completed = 1 WHERE id = ?", (task_id,))
        conn.commit()
        refresh_tasks()
    else:
        messagebox.showwarning("Warning", "Select a task to mark as completed")

def add_image():
    selected_item = task_listbox.curselection()
    if selected_item:
        task_id = int(task_listbox.get(selected_item).split(" - ")[0])
        file_path = filedialog.askopenfilename(filetypes=[("Image Files", ".png;.jpg;.jpeg;.gif")])
        if file_path:
            cursor.execute("UPDATE tasks SET image_path = ? WHERE id = ?", (file_path, task_id))
            conn.commit()
            refresh_tasks()
    else:
        messagebox.showwarning("Warning", "Select a task to add an image")

def refresh_tasks():
    task_listbox.delete(0, tk.END)
    cursor.execute("SELECT * FROM tasks")
    tasks = cursor.fetchall()
    for task in tasks:
        display_text = f"{task[0]} - {'✔' if task[2] else ''} {task[1]}"
        task_listbox.insert(tk.END, display_text)

def show_image():
    selected_item = task_listbox.curselection()
    if selected_item:
        task_id = int(task_listbox.get(selected_item).split(" - ")[0])
        cursor.execute("SELECT image_path FROM tasks WHERE id = ?", (task_id,))
        result = cursor.fetchone()
        
        if result and result[0]:
            image_path = result[0]
            if os.path.exists(image_path):
                img = Image.open(image_path)
                img.show()
            else:
                messagebox.showwarning("Warning", "Image file not found.")
        else:
            messagebox.showwarning("Warning", "No image associated with this task.")
    else:
        messagebox.showwarning("Warning", "Select a task to view the image")

def speech_to_text():
    """Captures speech input and adds it as a task."""
    recognizer = sr.Recognizer()  # Initialize recognizer
    
    with sr.Microphone() as source:
        print("Adjusting for background noise...")
        recognizer.adjust_for_ambient_noise(source, duration=1)
        print("Say something...")

        try:
            audio = recognizer.listen(source, timeout=5, phrase_time_limit=5)
            text = recognizer.recognize_google(audio)
            print("You said:", text)

            # Add recognized speech as a task
            add_task(text)
        except sr.UnknownValueError:
            messagebox.showwarning("Warning", "Could not understand, please try again.")
        except sr.RequestError:
            messagebox.showwarning("Warning", "Speech recognition service unavailable.")
        except Exception as e:
            messagebox.showwarning("Error", f"An error occurred: {e}")
            
def delete_all_tasks():
    cursor.execute("DELETE FROM tasks")  # Delete all tasks
    cursor.execute("DELETE FROM sqlite_sequence WHERE name='tasks'")  # Reset counter
    conn.commit()
    refresh_tasks()



# GUI Setup
root = tk.Tk()
root.title("To-Do List with Voice Input")
root.geometry("600x550")
root.configure(bg='#c8b2d8')

title_label = tk.Label(root, text="To-Do List", font=("Arial", 16, "bold"), bg="#f0f0f0")
title_label.pack(pady=10)

task_entry_frame = tk.Frame(root, bg="#f0f0f0")
task_entry_frame.pack(pady=5)

task_entry = tk.Entry(task_entry_frame, width=40, font=("Arial", 12))
task_entry.pack(side=tk.LEFT, padx=10)

add_button = tk.Button(task_entry_frame, text="Add", command=add_task, font=("Arial", 12), bg="#4CAF50", fg="white")
add_button.pack(side=tk.RIGHT)

task_listbox = tk.Listbox(root, width=60, height=15, font=("Arial", 12))
task_listbox.pack(pady=10)

button_frame = tk.Frame(root, bg="#f0f0f0")
button_frame.pack()

delete_button = tk.Button(button_frame, text="Delete", command=delete_task, font=("Arial", 12), bg="#B8255f", fg="white", width=12)
delete_button.grid(row=0, column=0, padx=5, pady=5)

complete_button = tk.Button(button_frame, text="Mark Completed", command=mark_completed, font=("Arial", 12), bg="#6ACCBC", fg="white", width=12)
complete_button.grid(row=0, column=1, padx=5, pady=5)

image_button = tk.Button(button_frame, text="Add Image", command=add_image, font=("Arial", 12), bg="#884DFF", fg="white", width=12)
image_button.grid(row=0, column=2, padx=5, pady=5)

show_image_button = tk.Button(button_frame, text="Show Image", command=show_image, font=("Arial", 12), bg="#808080", fg="white", width=12)
show_image_button.grid(row=0, column=3, padx=5, pady=5)

# 🎤 Speech-to-Text Button
speech_button = tk.Button(button_frame, text="🎤 Speak", command=speech_to_text, font=("Arial", 12), bg="#FFD700", fg="black", width=12)
speech_button.grid(row=2, column=1, padx=5, pady=10)
refresh_tasks()

delete_all_button = tk.Button(button_frame, text="Delete All", command=delete_all_tasks, font=("Arial", 12), bg="red", fg="white", width=12)
delete_all_button.grid(row=2, column=2, padx=5, pady=10)


# Close the database connection properly when the app exits
def on_closing():
    conn.close()
    root.destroy()

root.protocol("WM_DELETE_WINDOW", on_closing)
root.mainloop()


Adjusting for background noise...
Say something...
Adjusting for background noise...
Say something...
